In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Open this notebook in Google Colab with this link: 

# Introduction

This notebook illustrates how to use the *SC2 Academy* framework.

If you are using Google Colab, which is highly encouraged, please make sure to select a GPU runtime under: 

**'Runtime -> Change runtime type -> Hardware accelerator: GPU'**


Now run the cells below in the order they appear in!

# Setup

This installs all the packages needed to run PySC2 and TF-Agents.

In [ ]:
# since Colab does not have an X Server, we installing a virtual one
!pip install -q pysc2 pyvirtualdisplay
!apt-get install -y xvfb python-opengl mesa-utils libosmesa6-dev xorg x11-xserver-utils
!pip install --upgrade tf-agents==0.6.0
# to avoid conflicts, we first uninstall the pre-installed versions
!pip uninstall -y tensorboard
!pip uninstall -y tensorflow
!pip install tensorflow==2.3.0
!pip install tensorflow-probability==0.11.0
!pip install dm-reverb

!rm -R sample_data/ #free up some space

# Important - remove libtcmalloc
# This is the main roadblock stopping us originally from using Colab
# Credit goes to: https://medium.com/@n0mad/how-i-trained-starcraft-2-ais-using-googles-free-gpus-44bc635b0418
# Note that you will get a lot of errors looking like this
# ERROR: ld.so: object '/usr/lib/x86_64-linux-gnu/libtcmalloc.so.4' from LD_PRELOAD 
!apt-get remove libtcmalloc*

## Get the SC2 Academy

This pulls from the GitHub repository and moves the folders around for a simpler structure.

In [ ]:
!rm -R sc2_academy/
!git clone https://github.com/Ricechrispi/sc2_academy.git
!mv sc2_academy sc2_academy_
!mv sc2_academy_/sc2_academy sc2_academy
!rm -R sc2_academy_

## Download and Unpack StarCraft II

This first downloads and then unpacks the StarCraft II linux binary, which will take about ~6 minutes.

Note that unpacking requires to use the password ‘iagreetotheeula’. With this you agree to be bound by the terms of the **AI and Machine Learning License**. See https://github.com/Blizzard/s2client-proto#downloads

Downloading directly from Blizzard is quite slow. If you intend to use this notebook a lot, we recommend using a local Jupyter notebook instance or to copy the StarCraft II binary to your Google Drive and copy it here from there.

This could be done like this:
```
from google.colab import drive
drive.mount('/content/gdrive')
!cp /content/gdrive/My\ Drive/SC2.4.10.zip /content/SC2.4.10.zip 
drive.flush_and_unmount()
```



In [ ]:
import time
copy_time = 0
start_time = time.time()
!wget https://blzdistsc2-a.akamaihd.net/Linux/SC2.4.10.zip
copy_time += time.time() - start_time

unpack_time = 0
start_time = time.time()
!unzip -P iagreetotheeula -oq SC2.4.10.zip -d ~
unpack_time += time.time() - start_time

print(''.join(['copy_time = ', str(copy_time), ' unpack_time = ', str(unpack_time)]))

## Choose Training Hyper-Parameters

With these default values, an agent is trained for MoveToBeacon for ~2h on a normal Colab GPU instance. 


In [ ]:
#@title Hyper-Parameters for Trainig {run: 'auto'}
map_name = 'MoveToBeacon' #@param ['MoveToBeacon', 'CollectMineralShards', 'FindAndDefeatZerglings', 'DefeatRoaches', 'DefeatZerglingsAndBanelings', 'CollectMineralsAndGas', 'BuildMarines']
num_parallel_envs = 4 #@param {type:'slider', min:1, max:5, step:1}
num_environment_steps = 2000000  #@param {type:'slider', min:1000000, max:10000000, step:1000000}
num_collect_episodes = 20 #@param {type:'slider', min:12, max:32, step:4}
replay_buffer_max_length = 4000 #@param {type:'slider', min:1000, max:10000, step:1000}
use_lstms = False #@param {type:'boolean'}
misc_info = 'a short run' #@param {type: 'string'}

description = 'DESCRIPTION:' \
              + '\nmap_name: ' + map_name \
              + ';\num_parallel_envs: ' + str(num_parallel_envs) \
              + ';\nnum_environment_steps: ' + str(num_environment_steps) \
              + ';\num_collect_episodes: ' + str(num_collect_episodes) \
              + ';\nreplay_buffer_max_length: ' +str(replay_buffer_max_length) \
              + ';\nuse_lstms: ' + str(use_lstms) \
              + ';\nmisc_info: ' + misc_info


## Tensorboard

This sets up a TensorBoard instance to visualize the training progress.

NOTE: Execute this cell after every time you change the hyper-parameters. If the map name chagnes, the path for the tensorboard data also changes!

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir /content/results/$map_name

# Start Training

In [ ]:
# Start the training here. This cell could take multiple hours depending on the hyper-parameters.
!echo '$description' >> run_description.txt
!python sc2_academy/sc2_academy.py --map_name=$map_name --num_parallel_envs=$num_parallel_envs --num_environment_steps=$num_environment_steps --num_collect_episodes=$num_collect_episodes --replay_buffer_max_length=$replay_buffer_max_length --use_lstms=$use_lstms 2> run_output.log

# Evaluate

This evaluates your trained agent.

Important to note: 
*   The number of parallel environments
*   and wether you used LSTMs

must be the same as when training the agent. 
There is sadly no easy way for me to extract this information.

You can also choose to run a random evaluation, that creates an agent that takes random actions at every time step.




In [ ]:
#@title Hyper-Parameters for Evaluation {run: 'auto'}
eval_map_name = 'MoveToBeacon' #@param ['MoveToBeacon', 'CollectMineralShards', 'FindAndDefeatZerglings', 'DefeatRoaches', 'DefeatZerglingsAndBanelings', 'CollectMineralsAndGas', 'BuildMarines']
eval_num_parallel_envs = 4 #@param {type:'slider', min:1, max:5, step:1}
num_eval_episodes = 10 #@param {type:'slider', min:10, max:100, step:10}
use_lstms = False #@param {type:'boolean'}
misc_info = 'a short eval run' #@param {type: 'string'}

#@markdown --------------
#@markdown ### NOTE: If this is checked, a random agent is evaluated, NOT your trained agent 
random_eval = False #@param {type:'boolean'}

In [ ]:
# Start the evaluation here.
!python sc2_academy/sc2_academy.py --map_name=$eval_map_name --num_parallel_envs=$eval_num_parallel_envs --num_eval_episodes=$num_eval_episodes --use_lstms=$use_lstms --just_eval=True --just_eval_random=$random_eval 2> eval_output.log

# Export Results

This compresses all contents of the results folder and the log files into one file with a time stamp. 

After this is done, just download this file via **'right-click -> Download'** in the Colab file browser.

In [ ]:
import time
ts = time.gmtime()
time_string = time.strftime("%Y-%m-%d-%Hh-%Mm-%Ss", ts)

!zip -r /content/result_'$time_string'.zip /content/results /content/sc2_academy/ /content/run_output.log /content/run_description.txt /content/eval_output.log

# Clean-up

If you want to delete a (failed) run and start fresh, still small script deletes all the run-related files.

WARNING: this deletes _all_ the files in the results folder.

In [ ]:
!rm -R /content/results
!rm /content/result_*
!rm /content/run_output.log 
!rm /content/run_description.txt
!rm /content/eval_output.log